# NER to LOD

In diesem Notebook werden wir sehen, wie durch relativ kurzen Python-Code ein kurzer Text analysiert und die extrahierten Informationen mit Linked Open Data, am Beispiel der GND, verknüpft werden können.

Sie müssen dazu nichts zusätzlich installieren, sämtliche notwendige Software wir im Laufe des Notebooks in der Cloud heruntergeladen.

Bei diesem Dokument handelt es sich um ein sogenanntes Jupyter-Notebook. Jupyter-Notebooks erlauben das Ausführen von Python-Code in sogenannten Zellen. Daneben gibt es Text-Zellen wie diese. 

## Named Entity Recognition

In diesem Block annotieren wir einen kurzen Text mithilfe von spaCy, einer Python-Bibliothek für Textanalyse durch maschinelles Lernen. spaCy bietet sich gut an, da es wenig Anpassungen erfordert und bereits trainierte Modelle zur Verfügung stellt.

Bei spaCy wird ein Text in eine sogenannte Pipeline eingespiesen, wo der Text mehrere Schritte durchläuft:
pipeline.svg

Das erzeugte Doc-Object enthält dann den Text samt angereichter Informationen.

Führen Sie die folgenden Code-Zellen durch Klicken des Play-Buttons aus. Sie können die Zellen auch anpassen, z.B. um den Text zu verändern.

In [ ]:
# Wir speichern den zu annotierenden Texte unter der Variable "text"

text = "Aarau ist der Geburts- und Wohnort zahlreicher bekannter Persönlichkeiten. \
Als weltweit bekannteste Personen, die in Aarau geboren wurden, gelten der Ernährungswissenschaftler Max Bircher-Benner, \
der Erfinder des Birchermüesli, sowie Hans Herzog, General der Schweizer Armee."

In [ ]:
# Als nächstes laden wir ein Modell herunter, welches auf deutschen Texten trainiert wurde

!python -m spacy download de_core_news_sm

In [ ]:
# Wir müssen dann das Modell noch in Python laden
# Das Modell enthält die gesamte Pipeline! (Mehr Infos: https://spacy.io/models/de)

import spacy

nlp = spacy.load("de_core_news_sm")

In [ ]:
# Nun geben wir unseren Text in die Pipeline

doc = nlp(text)

In [ ]:
# Sehen wir uns den annotierten Text an!

for token in doc:
    print(token.text, token.ent_type_)

# Nicht angezeigt, aber auch vorhanden sind linguistische Informationen wie POS-tags und Dependenzen.

In [ ]:
# Wir können das auch hübscher darstellen dank des Moduls "displacy"

from spacy import displacy

displacy.render(doc, style='ent', jupyter=True)

So einfach können wir also einen Text annotieren!

Probieren Sie weitere Beispiele aus, indem sie den Text ersetzen, oder das spacy-Modell!

## Linking Data to Linked Open Databases

In diesem Abschnitt werden wir die vorher gefundenen Personen über die API von https://lobid.org suchen.

In [ ]:
# Unter dem Attribut "ents" des Doc-Objekts finden wir eine Liste aller erkannten Entitäten.

for ent in doc.ents:
    print(ent)

In [ ]:
import requests

for entity in doc.ents:

    # Mit diesem Befehl fragen wir die Lobid-API ab
    r = requests.get(f"https://lobid.org/gnd/search?q=preferredName%3A{entity}&format=json")

    result = r.json()

    # Für jedes Resultat geben wir die erste gefundene GND-URI heraus
    print(entity, "=>", result["member"][0]["id"])

Wir sehen hier schon, dass mit diesen einfachen Methoden relativ häufige Namen wie "Hans Herzog" nicht korrekt verlinkt werden. Wir würden unter den Resultaten zwar wahrscheinlich den richtigen finden, aber um eine Art "Ranking" zu erstellen, ist mehr Aufwand notwendig. 

Moderne Systeme verwenden für diese Verlinkung übrigens Machinelles Lernen, was weitaus besser funktioniert als der Abgleich einzelner Felder in einer Datenbank.

Im unteren Beispiel erweitere ich die Query noch um die Informationen des Berufs und des Geburtsorts von Hans Herzog, zwei Informationen, die relativ leicht automatisch im Text zu erkennen wären, hätte man ein entsprechendes Modell trainiert.

In [ ]:
import requests

# Wenn wir noch die Info hätten, dass Hans Herzog General war und in Aarau geboren wurde, könnten wir eine Query so schreiben:
query = "General Hans Herzog Aarau"

r = requests.get(f"https://lobid.org/gnd/search?q={query}&format=json")

result = r.json()

# Für jedes Resultat geben wir die erste gefundene GND-URI heraus
print(result["member"][0]["id"])

# Und es ist der Richtige!

https://d-nb.info/gnd/139764240


Eine sehr ähnliche Methodik, nur um einiges komplizierter, wird z.B. bei E-Periodica Next Level Access verwendet zur Verlinkung der Personen! (https://nla.e-periodica.ch/)